## Collecting Falcon 9 Launch Data via Webscraping

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

Creating functions to extract the data from the website's HTML table 

In [72]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]
def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])
    return out
def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD",table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass
def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    column_name=' '.join(row.contents)
    if not(column_name.strip().isdigit()):
        column_name=column_name.strip()
        return column_name

In [16]:
static_url="https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [26]:
response=requests.get(static_url)

In [36]:
soup=BeautifulSoup(response.content,"html.parser")

Finding all tables in the wikipedia page and extracting the one we want

In [41]:
html_tables=soup.find_all('table')

In [50]:
first_launch_table=html_tables[2]

In [68]:
column_names=[]
for row in first_launch_table.find_all('th'):
        columns=extract_column_from_header(row)
        if columns is not None and len(columns)>0:
            column_names.append(columns)
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Creating an empty dictionary and then filling it with data from the rows in the extracted table

In [69]:
launch_dict=dict.fromkeys(column_names)
del launch_dict['Date and time ( )']

In [70]:
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [75]:
extracted_row = 0
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False 
        row=rows.find_all('td')
        if flag:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            datatimelist=date_time(row[0])
            
            date=datatimelist[0].strip(',')
            launch_dict['Date'].append(date)

            time=datatimelist[1]
            launch_dict['Time'].append(time)
              
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
           
            launch_site=row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            payload=row[3].a.string
            launch_dict['Payload'].append(payload)
            
            payload_mass=get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            orbit=row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            customer=row[6].a.string if row[6].a else ""
            launch_dict['Customer'].append(customer)

            launch_outcome=list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            booster_landing=landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

In [78]:
webscrape_df=pd.DataFrame({key:pd.Series(value) for key, value in launch_dict.items()})

In [79]:
webscrape_df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,NASA,Success,F9 v1.0B0003.1,Failure,4 June 2010,18:45
2,2,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0004.1,No attempt\n,8 December 2010,15:43
3,3,CCAFS,Dragon,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0005.1,No attempt,22 May 2012,07:44
4,4,CCAFS,SpaceX CRS-1,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt\n,8 October 2012,00:35


In [80]:
webscrape_df.to_csv('spacex_web_scraped.csv',index=False)